# 第六章 处理输入: 链式 Prompt Chaining Prompts

 - [一、环境设置](#一、环境设置)
 - [二、 实现一个包含多个提示的复杂任务](#二、-实现一个包含多个提示的复杂任务)
     - [2.1 提取相关产品和类别名称](#2.1-提取相关产品和类别名称)
     - [2.2 检索提取的产品和类别的详细信息](#2.2-检索提取的产品和类别的详细信息)
     - [2.3 将 Python 字符串读取为 Python 字典列表](#2.3-将-Python-字符串读取为-Python-字典列表)
     - [2.4 根据详细的产品信息生成用户查询的答案](#2.4-根据详细的产品信息生成用户查询的答案)


在本章中，我们将学习如何通过将复杂任务拆分为一系列简单的子任务来链接多个 Prompt。

您可能会想，为什么要将任务拆分为多个 Prompt，而不是像我们在上一个视频中学习的那样，使用思维链推理一次性完成呢？我们已经证明了语言模型非常擅长遵循复杂的指令，特别是像 GPT-4 这样的高级模型。

那么让我们用两个比喻来解释为什么我们要这样做，来比较思维链推理和链式 Prompt。 

将任务拆分为多个 Prompt 的第一个比喻是一次性烹饪复杂菜肴与分阶段烹饪的区别。使用一个长而复杂的 Prompt 可能就像一次性烹饪复杂的菜肴，您必须同时管理多个成分、烹饪技巧和时间。这可能很具有挑战性，难以跟踪每个部分并确保每个组成部分都烹饪得恰到好处。另一方面，链式 Prompt 就像分阶段烹饪餐点，您专注于一个组成部分，确保每个部分都正确烹饪后再进行下一个。这种方法可以分解任务的复杂性，使其更易于管理，并减少错误的可能性。但是，对于非常简单的食谱，这种方法可能是不必要和过于复杂的。

一个稍微更好的比喻是，一次性完成所有任务与分阶段完成任务的区别。就像阅读一长串代码和使用简单的模块化程序之间的差异一样，复杂的依赖关系会导致代码变得混乱且难以调试。这个比喻同样适用于将复杂的单步任务提交给语言模型。当您有一个可以在任何给定点维护系统状态并根据当前状态采取不同操作的工作流程时，链式 Prompt 就成为一种强大的策略。

## 一、环境设置

同上一章，我们首先需要配置使用 OpenAI API 的环境

In [24]:
import json
from src.utils.chat_robot import OpenAiChat

## 二、 实现一个包含多个提示的复杂任务

### 2.1 提取相关产品和类别名称

在您对客户的查询进行分类后，您将获得查询的类别——是账户问题还是产品问题。然后您可以根据不同的类别采取不同的行动。

每个子任务仅包含执行对应任务所需的指令，这使得系统更易于管理，确保模型具备执行任务所需的所有信息，并降低了出错的可能性。这种此方法还可以降低成本，因为更长的 Prompt 和更多的 tokens 会导致更高的运行成本，并且在某些情况下可能不需要概述所有步骤。

这种方法的另一个好处是，它更容易测试哪些步骤可能更容易失败，或者在特定步骤中需要人工干预。

随着您与这些模型的构建和交互不断深入，您将逐渐培养出何时运用此策略的直觉。另外，还有一个额外的好处是，它允许模型在必要时使用外部工具。例如，它可能决定在产品目录中查找某些内容，调用 API 或搜索知识库，这是使用单个 Prompt 无法实现的。



In [29]:
delimiter = "####"
system_message = f"""
你将提供查询服务。
查询服务使用{delimiter}字符分隔。

输出一个json列表，不包含其他字符信息，每个元素具有以下格式：
    'category': <计算机和笔记本电脑、智能手机和配件、电视和家庭影院系统、游戏机和配件、音频设备、相机和摄像机中的一个>,
    'products': <必须在下面的允许产品列表中找到的产品列表>
    或
    'category': <计算机和笔记本电脑、智能手机和配件、电视和家庭影院系统、游戏机和配件、音频设备、相机和摄像机中的一个>,

类别和产品必须在查询服务中找到。
在产品和类别中进行模糊匹配，如果提及了产品，输出相应的产品和类别；如果没有提及产品但提及了类别，输出相应的类别；如果未找到产品或类别，则输出空列表。

允许的产品：

计算机和笔记本电脑类别：
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

智能手机和配件类别：
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds

电视和家庭影院系统类别：
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

游戏机和配件类别：
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

音频设备类别：
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

相机和摄像机类别：
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera
"""
query = "请查询 SmartX ProPhone 智能手机和 FotoSnap 相机，包括单反相机。另外，请查询关于电视产品的信息。"
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': f"{delimiter}{query}{delimiter}"},
] 

category_and_product = OpenAiChat().get_completion_from_messages(messages)
print(category_and_product)

[
    {
        "category": "智能手机和配件",
        "products": [
            "SmartX ProPhone"
        ]
    },
    {
        "category": "相机和摄像机",
        "products": [
            "FotoSnap DSLR Camera",
            "FotoSnap Mirrorless Camera"
        ]
    },
    {
        "category": "电视和家庭影院系统"
    }
]


正如您所见，对于我们的输出是一个对象列表，每个对象都有一个类别和一些产品。如"SmartX ProPhone"和"Fotosnap DSLR Camera"

在最后一个对象中，我们只有一个类别，因为没有提到任何具体的电视。

这种结构化的响应输出的好处是可以轻松地将其读入 Python 的列表中。

让我们尝试另一个例子。

In [18]:
user_message_2 = "我的路由器坏了"
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': f"{delimiter}{user_message_2}{delimiter}"},
] 
response = OpenAiChat().get_completion_from_messages(messages)
print(response)

[]


如果您留意列表，会发现我们实际上并没有包含任何路由器的信息。

现在，我们需要对其进行正确的格式化以完成输出。

正如您所见，在这种情况下，输出是一个空列表。

### 2.2 检索提取的产品和类别的详细信息

我们提供大量的产品信息作为示例，要求模型提取产品和对应的详细信息

In [19]:
# 产品信息
from tools import PRODUCTS
products = PRODUCTS

def get_product_by_name(name):
    """
    根据产品名称获取产品

    参数:
    name: 产品名称
    """
    return products.get(name, None)

def get_products_by_category(category):
    """
    根据类别获取产品

    参数:
    category: 产品类别
    """
    return [product for product in products.values() if product["category"] == category]

In [20]:
print(get_product_by_name("TechPro Ultrabook"))

{'name': 'TechPro 超极本', 'category': '电脑和笔记本', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'], 'description': '一款时尚轻便的超极本，适合日常使用。', 'price': 799.99}


In [21]:
print(get_products_by_category("电脑和笔记本"))

[{'name': 'TechPro 超极本', 'category': '电脑和笔记本', 'brand': 'TechPro', 'model_number': 'TP-UB100', 'warranty': '1 year', 'rating': 4.5, 'features': ['13.3-inch display', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'], 'description': '一款时尚轻便的超极本，适合日常使用。', 'price': 799.99}, {'name': 'BlueWave 游戏本', 'category': '电脑和笔记本', 'brand': 'BlueWave', 'model_number': 'BW-GL200', 'warranty': '2 years', 'rating': 4.7, 'features': ['15.6-inch display', '16GB RAM', '512GB SSD', 'NVIDIA GeForce RTX 3060'], 'description': '一款高性能的游戏笔记本电脑，提供沉浸式体验。', 'price': 1199.99}, {'name': 'PowerLite Convertible', 'category': '电脑和笔记本', 'brand': 'PowerLite', 'model_number': 'PL-CV300', 'warranty': '1 year', 'rating': 4.3, 'features': ['14-inch touchscreen', '8GB RAM', '256GB SSD', '360-degree hinge'], 'description': '一款多功能的可转换笔记本电脑，具有灵敏的触摸屏。', 'price': 699.99}, {'name': 'TechPro Desktop', 'category': '电脑和笔记本', 'brand': 'TechPro', 'model_number': 'TP-DT500', 'warranty': '1 year', 'rating': 4.4, 'features': ['Intel Core i7 proc

### 2.3 将 Python 字符串读取为 Python 字典列表

#### 2.3.1 召回相关产品和类别的详细信息

In [33]:
def read_string_to_list(input_string):
    """
    将输入的字符串转换为 Python 列表。

    参数:
    input_string: 输入的字符串，应为有效的 JSON 格式。

    返回:
    list 或 None: 如果输入字符串有效，则返回对应的 Python 列表，否则返回 None。
    """
    if input_string is None:
        return None

    try:
        # 将输入字符串中的单引号替换为双引号，以满足 JSON 格式的要求
        input_string = input_string.replace("'", "\"")
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None

def generate_output_string(data_list):
    """
    根据输入的数据列表生成包含产品或类别信息的字符串。

    参数:
    data_list: 包含字典的列表，每个字典都应包含 "products" 或 "category" 的键。

    返回:
    output_string: 包含产品或类别信息的字符串。
    """
    output_string = ""
    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string 

In [34]:
category_and_product_list = read_string_to_list(category_and_product)
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

{
    "name": "SmartX ProPhone",
    "category": "智能手机和配件",
    "brand": "SmartX",
    "model_number": "SX-PP10",
    "warranty": "1 year",
    "rating": 4.6,
    "features": [
        "6.1-inch display",
        "128GB storage",
        "12MP dual camera",
        "5G"
    ],
    "description": "一款拥有先进摄像功能的强大智能手机。",
    "price": 899.99
}
{
    "name": "FotoSnap DSLR Camera",
    "category": "相机和摄像机",
    "brand": "FotoSnap",
    "model_number": "FS-DSLR200",
    "warranty": "1 year",
    "rating": 4.7,
    "features": [
        "24.2MP sensor",
        "1080p video",
        "3-inch LCD",
        "Interchangeable lenses"
    ],
    "description": "使用这款多功能的单反相机，捕捉惊艳的照片和视频。",
    "price": 599.99
}
{
    "name": "FotoSnap Mirrorless Camera",
    "category": "相机和摄像机",
    "brand": "FotoSnap",
    "model_number": "FS-ML100",
    "warranty": "1 year",
    "rating": 4.6,
    "features": [
        "20.1MP sensor",
        "4K video",
        "3-inch touchscreen",
        "Interchangeable lens

### 2.4 根据详细的产品信息生成用户查询的答案

In [42]:
system_message = f"""
你是一家大型电子商店的客服助理。
请以友好和乐于助人的口吻回答问题，并尽量简洁明了。
最后请确保向用户询问相关的后续问题。
"""
user_message_1 = f"""
请介绍一下 SmartX ProPhone 智能手机和 FotoSnap 相机，包括单反相机。
另外，介绍关于电视产品的信息。"""
messages = [
    {'role': 'system', 'content': system_message},
    {'role': 'user', 'content': user_message_1},
    {'role': 'assistant', 'content': f"""相关产品信息:\n\
 {product_information_for_user_message_1}"""},
]
final_response = OpenAiChat().get_completion_from_messages(messages)
print(final_response)

SmartX ProPhone 是一款功能强大的智能手机，拥有6.1英寸显示屏、128GB存储、12MP双摄像头和5G网络支持，售价为899.99美元。FotoSnap 单反相机包括 DSLR 相机和无反相机，分别具有不同的功能和特点，价格分别为599.99美元和799.99美元。

关于电视产品，我们有多款选择，包括 CineView 4K 电视、CineView 8K 电视和 CineView OLED 电视，分别具有不同的分辨率和功能，价格从599.99美元到2999.99美元不等。此外，我们还有 SoundMax 家庭影院系统和 Soundbar，可提供卓越的音频体验，价格从199.99美元到399.99美元不等。

您对哪款产品感兴趣？您想了解更多详细信息或者有其他问题吗？


我们讨论了如何通过一系列步骤加载与用户查询相关的信息，为模型提供所需的上下文，以有效回答问题。

您可能会想，为什么我们选择性地将产品描述加载到提示中，而不是包含所有产品描述，让模型使用它所需的信息呢？

这其中有几个原因。

首先，包含过多的产品描述可能会使模型在处理上下文时感到困惑，就像对于试图一次处理大量信息的人一样。当然，对于像 GPT-4 这样更高级的模型来说，这个原因就不太重要了。尤其是当上下文像这个例子一样具有良好的结构时，模型足够聪明，能够巧妙地忽略那些明显不相关的信息。

接下来的原因更加具有说服力。

首先，包含所有产品描述可能会使模型对上下文更加混乱，就像对于试图一次处理大量信息的人一样。当然，对于像 GPT-4 这样更高级的模型来说，这个问题不太相关，特别是当上下文像这个例子一样结构良好时，模型足够聪明，只会忽略明显不相关的信息。接下来的原因更有说服力。

第二个原因是，语言模型有上下文限制，即固定数量的 token 允许作为输入和输出。如果您有一个巨大的产品目录，您甚至无法将所有描述都放入上下文窗口中。

最后一个原因是，包含所有产品描述可能会使模型过拟合，因为它会记住所有的产品描述，而不是只记住与查询相关的信息。这可能会导致模型在处理新的查询时表现不佳。

使用语言模型时，由于按 token 付费，可能会很昂贵。因此，通过有选择地加载信息，可以减少生成响应的成本。一般来说，确定何时动态加载信息到模型的上下文中，并允许模型决定何时需要更多信息，是增强这些模型能力的最佳方法之一。

并且要再次强调，您应该将语言模型视为需要必要上下文才能得出有用结论和执行有用任务的推理代理。因此，在这种情况下，我们必须向模型提供产品信息，然后它才能根据该产品信息进行推理，为用户创建有用的答案。

在这个例子中，我们只添加了一个特定函数或函数的调用，以通过产品名称获取产品描述或通过类别名称获取类别产品。但是，模型实际上擅长决定何时使用各种不同的工具，并可以正确地使用它们。这就是 ChatGPT 插件背后的思想。我们告诉模型它可以访问哪些工具以及它们的作用，它会在需要从特定来源获取信息或想要采取其他适当的操作时选择使用它们。在这个例子中，我们只能通过精确的产品和类别名称匹配查找信息，但还有更高级的信息检索技术。检索信息的最有效方法之一是使用自然语言处理技术，例如命名实体识别和关系提取。

另一方法是使用文本嵌入（Embedding）来获取信息。嵌入可以用于实现对大型语料库的高效知识检索，以查找与给定查询相关的信息。使用文本嵌入的一个关键优势是它们可以实现模糊或语义搜索，这使您能够在不使用精确关键字的情况下找到相关信息。因此，在此例子中，我们不一定需要产品的确切名称，而可以使用更一般的查询如 **“手机”** 进行搜索。我们计划很快推出一门全面的课程，介绍如何在各种应用中使用嵌入，敬请关注。

在下一章中我们将讨论如何评估语言模型的输出。